### APOIO AO GOVERNO DA BANCADA PARTIDARIA POR VOTO PARLAMENTAR
    
Ranking dos partidos agregado por político.

Atualmente usando 

$DEMA = 2MA - MA(MA)$

onde $MA$ é uma moving average, com os parametros:

halflife = 90

min_periods = 20

#### OUTPUT

- Gráfico Apoio Ao Governo Da Bancada Partidaria por Voto Parlamentar
- CSV Apoio Governo Deputado
- CSV Id Proposicao com Orientacao do Governo

In [16]:
import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd
import glob
import os
import pickle
import seaborn as sns
from copy import deepcopy
import yaml

%matplotlib inline  
%pylab inline
pylab.rcParams['figure.figsize'] = (20,12)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning:

pylab import has clobbered these variables: ['text', 'f']
`%matplotlib` prevents importing * from pylab and numpy



#### CONECTA AO BANCO

In [50]:
with open('server_config.yaml', 'r') as f:
        server = yaml.load(f)

host = server['host']
database = server['database']
user = server['user']
password = server['password']


url = 'postgresql://{}:{}@{}/{}'
url = url.format(user, password, host, database)
conn = sqlalchemy.create_engine(url)

##### CONECTA PLOTLY

In [51]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username=server['plotly_username_1'], api_key=server['plotly_key_1'])
#plotly.tools.set_credentials_file(username=server['plotly_username_2'], api_key=server['plotly_key_2'])

#### MOVING AVERAGES

In [54]:
#### Double Moving Average
def dema(series, halflife=30, min_periods=10):
    
    ma = series.ewm(halflife=halflife, min_periods=min_periods).mean()
    
    dema = 2*ma - ma.ewm(halflife=halflife).mean()
    
    return dema

#### TRIPLE MOVING AVERAGE
def tema(series, halflife=30):
    
    ma = series.ewm(halflife=halflife).mean()
    
    dma = ma.ewm(halflife=halflife).mean()
    
    tma = 3*ma - 3*dma + dma.ewm(halflife=halflife).mean()
    
    return tma

#### INFOS GERAIS

In [55]:
relacoes = {'SD': ['SDD', 'SOLIDARIED', 'SD'],
           'GOV': ['APOIO AO GOVERNO', 'GOV', 'GOV.'],
           'DEM': ['DEM', 'PFL'],
           'PR': ['PRONA', 'PL', 'PR', 'PST'],
           'PTB': ['PAN', 'PTB'],
           'PRB': ['PRB', 'PMR'],
           'PP': ['PP', 'PPB']}

color_pec ='#2980b9'
color_pl = '#c0392b'

In [56]:
def substitui_sigla(sigla):
    
    for final, values in relacoes.items():
        if sigla.upper() in values:
            return final
    else:
        return sigla.upper()

##### Load votações dos deputados e votacao_id_table

In [57]:
fidelidade_deputado = pd.read_sql_query('SELECT * FROM desenv.votacao_governo_deputados_dia ORDER BY datavotacao DESC',
                                        conn)
votacao_id_table = pd.read_sql_query('SELECT * FROM desenv.id_proposicao_com_orient_governo',
                                        conn)

#### Por voto do deputado no partido

In [58]:
from collections import OrderedDict
partidos_todos =  OrderedDict( 
                    [('PMDB',{'cor': '#EA642A'}),
                    ('PT',{'cor': '#D81224'}),
                    ('PSDB',{'cor': '#1F5393'}),
                    ('PSB',{'cor': '#F7AF32'}),
                    ('DEM', {'cor':'#1B8396'}),
                    ('PTB', { 'cor':'#27B8E8'}),
                    ('PDT', { 'cor':'#9D8577'}),
                    ('PP', { 'cor':'#CCBE70'}),
                    ('PPS', { 'cor':'#9374AE'}),
                    ('PV', { 'cor':'#69963D'}),
                    ('PR', { 'cor':'#DAA05C'}),
                    ('PCdoB', { 'cor':'#BE4060'}),
                    ('PSC', { 'cor':'#F9CCAF'}),
                    ('PRB', { 'cor':'#5D879D'}),
                    ('PSOL', { 'cor':'#FCDB3A'}),
                   # ('PSTU', { 'cor':'#EB757F'}),
                    ('PTC', { 'cor':'#E1DBB8'}),
                    ('PMN', { 'cor':'#69C3CE'}),
                    ('PRP', { 'cor':'#D2D63A'}),
                    ('PTdoB', { 'cor':'#7D1317'}),
                    ('PCB', { 'cor':'#B8452C'}),
                    ('PRTB', { 'cor':'#B18EBD'}),
                    ('PHS', { 'cor':'#9FB292'}),
                    ('PSDC', { 'cor':'#109745'}),
                    ('PCO', { 'cor':'#575656'}),
                    ('PTN', { 'cor':'#218CCD'}),
                    ('PSL', { 'cor':'#E5E68E'}),
                    ('PSD', { 'cor':'#803367'}),
                    ('PPL', { 'cor':'#B3B2B2'}),
                    ('PEN', { 'cor':'#9F742E'}),
                    ('PROS', { 'cor':'#E63D85'}), 
                    ('REDE', {'cor': '#3AB6C1'})])

In [59]:
fidelidade_deputado.head(1)

,index,codproposicao_x,tipo_x,numero,ano,codsessao,datavotacao,idecadastro,nome,partido,uf,voto,orientacao,votacao_id,fidelidade,quorum,partido_atualizado,tam_bancada
0,0,2121837,MPV,755,2016,16212,2017-05-17 19:23:00,133439.0,André Figueiredo,PDT,CE,Não,Não,9154465944888420931,1,269,PDT,10


In [60]:
# SUBSTITUI OS PARTIDOS QUE SE UNIRAM OU MUDARAM DE SIGLA
fidelidade_deputado['partido_atualizado'] = fidelidade_deputado['partido'].apply(substitui_sigla) 

In [61]:
# CALCULA MEDIA GERAL
votos_total = fidelidade_deputado[['votacao_id', 'fidelidade']].groupby(['votacao_id']).sum()
quorum_total     = fidelidade_deputado[['votacao_id', 'fidelidade']].groupby(['votacao_id']).count().rename(columns={'fidelidade': 'quorum_total'})
fidelidade_total = votos_total['fidelidade'].divide(quorum_total['quorum_total'])

In [62]:
fidelidade_deputado_partido = fidelidade_deputado.groupby(
                                        ['partido_atualizado', 'votacao_id']).apply(
                                                lambda x: sum(x['fidelidade'])/x['tam_bancada'].unique()[0])

In [63]:
fidelidade_deputado_partido = fidelidade_deputado_partido.to_frame().reset_index().pivot(index='votacao_id', columns='partido_atualizado', values=0)

In [64]:
fidelidade_deputado_partido = pd.concat([fidelidade_deputado_partido,fidelidade_total], axis=1).rename(columns={0:'Apoio Médio'})

In [65]:
fidelidade_deputado_partido = fidelidade_deputado_partido.reset_index().merge(votacao_id_table[['votacao_id', 'datavotacao']], on='votacao_id')
fidelidade_deputado_partido.index = fidelidade_deputado_partido['datavotacao']
del fidelidade_deputado_partido['datavotacao']
del fidelidade_deputado_partido['votacao_id']
fidelidade_deputado_partido.sort_index(inplace=True)

In [66]:
### FILTRA OS PARTIDOS POR PERIODO

def cut_series(series, timestamp):       
    a = series[:timestamp].apply(lambda x: np.nan)
    b = series[timestamp:]
    return a.append(b)

partidos_filtrados = [('PSDC', '2014-06-01'),
                      ('PTN', '2015-02-01'),
                      ('PHS', '2007-03-01'),
                      ('PSL', '2011-01-01'),
                      ('PMN', '2007-01-01'),
                      ('PSC', '2003-01-01'),
                      ('PSD', '2012-01-01')]

for partido, timestamp in partidos_filtrados:
    
    fidelidade_deputado_partido[partido] = cut_series(fidelidade_deputado_partido[partido], timestamp)

In [67]:
## RESCALE TO [-1 to 1]

fidelidade_deputado_partido = fidelidade_deputado_partido.applymap(lambda x: 2*x - 1)

In [69]:
data = []


halflife = 90
min_periods = 20
# MEDIA GERAL
partido = 'Apoio Médio'
d = go.Scatter(
                name = '{0}'.format(partido),
                mode = 'lines',
                line = dict(
                            color = '#7F7F7F',
                            width = 2),

                x = fidelidade_deputado_partido.index, 
                y = dema(fidelidade_deputado_partido[partido], halflife=halflife, min_periods=min_periods)
        )
data.append(d)


#### MEDIA FIXA 60 VOTACOES
for partido, args in partidos_todos.items():
    
    try:
        y = dema(fidelidade_deputado_partido[partido.upper()], halflife=halflife, min_periods=min_periods)
    except:
        continue
    d = go.Scatter(
                visible = "legendonly",
                name = '{0}'.format(partido.upper()),
                mode = 'lines',
                line = dict(
                            color = (args['cor']),
                            width = 2),
                x = fidelidade_deputado_partido.index,
                y = y
        )
    data.append(d)

#### RETANGULOS PRESIDENTES
rectangle_color = '#ECEFF0'
opacity = 0.8
presidentes = [
        # LULA I
        {
            'type': 'rect','xref': 'x','yref': 'y','x0': '2003-01-01','y0': -1.1,'x1': '2006-12-31','y1': 2,
            'fillcolor':rectangle_color ,'opacity': opacity ,'line': {
                'width': 0
            }, 'layer':'below',
        },
        # DILMA I
        {
            'type': 'rect','xref': 'x','yref': 'y','x0': '2011-01-01','y0': -1.1,'x1': '2014-12-31','y1': 2,
            'fillcolor': rectangle_color,'opacity': opacity ,'line': {
                'width': 0
            },'layer':'below'
        },
        # TEMER 
        {
            'type': 'rect','xref': 'x','yref': 'y','x0': '2016-05-01','y0': -1.1,
            'x1': datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], '%Y-%m-%d'),
            'y1': 2,
            'fillcolor': rectangle_color,'opacity': opacity ,'line': {
                'width': 0,
            }, 'layer':'below'
        }]  

### ANOTACOES PRESITENTES
y_pred = 1.05
presidentes_hist = [dict(x='2000-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='FHC II',
                      ax=0, ay=-30,
                      showarrow=False,
                        font=dict(size=9)),
                    dict(x='2004-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='LULA I',
                      ax=0, ay=-30,
                      showarrow=False,
                        font=dict(size=9)),
                   dict(x='2008-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='LULA II',
                      ax=0, ay=-30,
                      showarrow=False,
                       font=dict(size=9)),
                   dict(x='2012-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='DILMA I',
                      ax=0, ay=-30,
                      showarrow=False,
                       font=dict(size=9)),
                   dict(x='2015-08-15',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='DILMA II',
                      ax=0, ay=-30,
                      showarrow=False,
                       font=dict(size=9)),
                   dict(x='2016-12-01',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='TEMER',
                      ax=0, ay=-30,
                      showarrow=False,
                      font=dict(size=9))]

#### RETANGULOS PRESIDENTE CAMARA
rectangle_color = '#ECEFF0'
opacity = 0.8
shape_skeleton = {'type': 'rect','xref': 'x','yref': 'y','x0': '2003-01-01','y0': -1.1,'x1': '2006-12-31','y1': 2,
            'fillcolor':rectangle_color ,'opacity': opacity ,'line': {'width': 0}, 'layer':'below'}


presidentes_camara = [ {'mandato': ('1999-02-02','2001-02-13'), 'nome': 'TEMER', 'partido':'PMDB'},
                       {'mandato': ('2001-02-13','2002-12-17'), 'nome': 'AÉCIO', 'partido':'PSDB'},
                       {'mandato': ('2003-02-02','2005-02-14'), 'nome': 'LIMA'  , 'partido':'PT'},
                       {'mandato': ('2005-02-14','2005-07-21'), 'nome': '*', 'partido':''},
                       {'mandato': ('2005-07-28','2007-02-01'), 'nome': 'REBELO', 'partido':'PCdoB'},
                       {'mandato': ('2007-02-01','2009-02-02'), 'nome': 'CHINAGLIA' , 'partido':'PT'},
                       {'mandato': ('2009-02-02','2010-12-17'), 'nome': 'TEMER', 'partido':'PMDB'  },
                       {'mandato': ('2010-12-17','2013-02-04'), 'nome': 'AURELI', 'partido':'PT'},
                       {'mandato': ('2013-02-04','2015-02-01'), 'nome': 'ALVES', 'partido':'PMDB'},
                       {'mandato': ('2015-02-01','2016-07-07'), 'nome': 'CUNHA', 'partido':'PMDB'},
                       {'mandato': ('2016-07-07',
                                   datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], '%Y-%m-%d')),
                      'nome': 'MAIA', 'partido':'DEM'}]


presidentes_camara_shapes = []
for i in range(len(presidentes_camara)):
    if i % 2 != 0:
        shape_skeleton.update(dict(zip(['x0','x1'],presidentes_camara[i]['mandato'])))
        presidentes_camara_shapes.append(deepcopy(shape_skeleton))

#### ANOTACOES PRESIDETES CAMARA

y_pred = 1.05
annot_skeleton = dict(x='2000-12-31', y=y_pred, xref='x', yref='paper', text='FHC II',ax=0, ay=-30,
                     showarrow=False,
                       font=dict(size=9))

presidentes_camara_annot = []
for presidente in presidentes_camara:
    a, b  = presidente['mandato']
    a = datetime.datetime.strptime(a, '%Y-%m-%d')
    b = datetime.datetime.strptime(b, '%Y-%m-%d')
    mean = a + (b - a) /2
    annot_skeleton.update(dict(zip(['x','text'], (mean, '{0}<br>{1}'.format(presidente['nome'], presidente['partido'])))))
    presidentes_camara_annot.append(deepcopy(annot_skeleton))
    
severino = [dict(text='* SEVERINO PP', font=dict(size=9), x=0.5, y=-0.47, yref='paper', xref='paper', showarrow=False)]
    


### ACONTECIMENTOS
acontecimentos = [dict(x='2013-06-23',
                      y=1,
                      xref='x', yref='paper',
                      text='Jornadas de <br> Junho',
                      ax=0, ay=-18,
                      font=dict(size=10)),
               dict(x='2005-06-30',
                    y=1,
                    xref='x', yref='paper',
                    text='Mensalão',
                    ax=0, ay=-12,
                   font=dict(size=10)),
                dict(x='2008-09-15',
                    y=1,
                    xref='x', yref='paper',
                    text='Crise <br> Financeira',
                    ax=0, ay=-18,
                    font=dict(size=10)),
                dict(x='2016-05-12',
                    y=1,
                    xref='x', yref='paper',
                    text='Impeachment',
                    ax=0, ay=-12,
                    font=dict(size=10)),]
                    

acontecimentos_lines =  [{
                            'type': 'line',
                            'x0': ac['x'],
                            'y0': -1.1,
                            'x1': ac['x'],
                            'y1': 1.1,
                            'line': {
                                'color': 'black',
                                'width': 1,
                            },
                        } for ac in acontecimentos]


y_arrows = [dict(x=-0.012, y=1.02, ax=0, ay=17, arrowsize=1, arrowhead=4, xref='paper' )]

### TEXTO DO GRAFICO
subtitulos = [dict(text='PRESIDENTES', font=dict(size=11), x=0, y=-0.12, yref='paper', xref='paper', showarrow=False),
             dict(text='INFORMAÇÕES', font=dict(size=11), x=0, y=-0.32, yref='paper', xref='paper', showarrow=False),
             dict(text='PARTIDOS', font=dict(size=11), x=0, y=-0.52, yref='paper', xref='paper', showarrow=False),
             dict(text='Dados: <a href=\"http://www2.camara.leg.br/transparencia/dados-abertos/dados-abertos-legislativo\">Câmara dos Deputados</a>', 
                  font=dict(size=9), x=0, y=-0.95, yref='paper', xref='paper', showarrow=False),
             dict(text='Atualizado: {}'.format(datetime.datetime.strftime(datetime.datetime.now(), '%d/%m/%Y')), 
                  font=dict(size=9), x=0.35, y=-0.95, yref='paper', xref='paper', showarrow=False),
             dict(text='Elaborado por Congresso Em Números', 
                  font=dict(size=9), x=0.60, y=-0.95, yref='paper', xref='paper', showarrow=False),]


#### BOTOES PRESIDENTES
y_button_president = -0.22
updatemenus = list([
    dict(buttons=list([   
            dict(label = 'FHC II',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=[datetime.datetime.strftime(fidelidade_deputado_partido.index[0], 
                                                                          '%Y-%m-%d'), 
                                               '2002-12-31'])}]),  
           
            dict(label = 'LULA I',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2003-01-01', '2006-12-31'])}]),

            dict(label = 'LULA II',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2007-01-01', '2010-12-31'])}]),

            dict(label = 'DILMA I',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2011-01-01', '2014-12-31'])}]),

            dict(label = 'DILMA II',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2015-01-01', '2016-05-01'])}]),
        
            dict(label = 'TEMER',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2016-05-01', 
                                               datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], 
                                                                          '%Y-%m-%d')])}]),
             dict(label = 'TODOS',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=[datetime.datetime.strftime(fidelidade_deputado_partido.index[0], 
                                                                          '%Y-%m-%d'), 
                                               datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], 
                                                                          '%Y-%m-%d')])}]),
            ]),
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = True,
            active = 6,
            type = 'buttons',
            x = 0,
            xanchor = 'left',
            y = y_button_president,
            yanchor = 'bottom' ),

    ## BOTOES INFORMACOES
    dict(buttons=list([   
        dict(label = 'Acontecimentos',
             method = 'relayout',
             args = [{'annotations': acontecimentos +  subtitulos,
                     'shapes': acontecimentos_lines,
                     'xaxis': dict(tickvals=[x['x'] for x in acontecimentos])}]),
        dict(label = 'Presidentes',
             method = 'relayout',
             args = [{'annotations': presidentes_hist + subtitulos,
                     'shapes': presidentes,
                     'xaxis': dict(tickvals=['2003', '2007', '2011', '2015'], tickformat='%Y')}]),
        dict(label = 'Presidentes Câmara',
             method = 'relayout',
             args = [{'annotations': presidentes_camara_annot + subtitulos + severino,
                     'shapes': presidentes_camara_shapes,
                     'xaxis': dict(tickvals=['2001','2003','2005', '2007','2009', '2011', '2013', '2015', '2007', '2017'], tickformat='%Y')}]),
         dict(label = 'Limpar',
             method = 'relayout',
             args = [{'annotations': subtitulos,
                     'shapes': [],
                      'xaxis': dict(tickvals=['2003', '2007', '2011', '2015'], tickformat='%Y')}]),
        ]),
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = True,
            active = 3,
            type = 'buttons',
            x = 0,
            xanchor = 'left',
            y = -0.42,
            yanchor = 'bottom' )
])


legend = dict(orientation='h', x=0, y=-0.72, xanchor='center', yanchor='middle')

yaxis = go.YAxis(
    #title="Apoio ao Governo",
    range=[-1.1, 1.1],
    showgrid=False,
    showline=True,
    ticks="", 
    showticklabels=True,
    #mirror=True,
    linewidth=1,
    ticktext=['Menos<br>Apoio', 'Mais<br>Apoio'],
    tickvals=[-0.96, 1.09]
)
layout = dict(title = 'Apoio ao Governo das Bancadas Partidárias por Voto <br> Parlamentar na Câmara ',
              xaxis = dict(#rangeslider=dict(),
                            type='date'),
              yaxis = yaxis,
              autosize=False,
              width=600,
              height=880,
              margin=go.Margin(
                    l=50,
                    r=30,
                    b=360,
                    t=110,
                    pad=0
                ),
              paper_bgcolor='#F2F2F2',
              plot_bgcolor='#F2F2F2',
              updatemenus=updatemenus,
              annotations=subtitulos,
              legend=legend,
              font = dict(family='Lato, sans-serif')
              #shapes = presidentes,
)

            
fig = dict(data = data, layout=layout)
py.iplot(fig, filename='Apoio Governo Bancadas Voto Parlamentar')

The draw time for this plot will be slow for clients without much RAM.


/usr/local/lib/python3.5/dist-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [70]:
### TO SQL
fidelidade_deputado_partido.apply(dema).to_sql('camdep_apoio_governo_bancada_voto_parlamentar', conn, schema='indicadores', if_exists='replace')

#### Output CSVs

In [71]:
fidelidade_deputado.to_csv('csvs/votacao_governo_deputado', index='False', encoding='utf8')
votacao_id_table.to_csv('csvs/id_proposicao_governo', index='False', encoding='utf8')
fidelidade_deputado_partido.to_csv('csvs/fidelidade_deputado_partido_raw', index='False', encoding='utf8')
fidelidade_deputado_partido.apply(dema).to_csv('csvs/fidelidade_deputado_partido_dema', index='False', encoding='utf8')

In [72]:
"""
STACKED CHART


votos_apoio = fidelidade_deputado.groupby(['votacao_id', 'partido']).sum()['fidelidade'].to_frame().reset_index()
a = fidelidade_deputado.drop_duplicates(['votacao_id', 'partido'])[['votacao_id', 'partido', 'tam_bancada']]
votos = a.merge(votos_apoio, on=['votacao_id', 'partido'])
votos['contra'] = votos.apply(lambda x:  x['tam_bancada'] - x['fidelidade'], axis=1)
votos = votos.rename(columns= {'fidelidade': 'a_favor'})
votos_favor = votos.pivot(index='votacao_id', columns='partido', values='a_favor')
votos_contra = votos.pivot(index='votacao_id', columns='partido', values='contra')
def add_date(df):
    df = df.reset_index().merge(votacao_id_table[['votacao_id', 'datavotacao']], on='votacao_id')
    df.index = df['datavotacao']
    del df['datavotacao']
    del df['votacao_id']
    df.sort_index(inplace=True)
    return df

votos_contra = add_date(votos_contra)
votos_favor  = add_date(votos_favor)
votos_contra = votos_contra.fillna(0)
def stack(df):
    
    for i, column in enumerate(df.columns):
        if i == 0:
            continue
        df[column] = df[column] + df[df.columns[i-1]]
        
    return df
votos_contra_stack = stack(votos_contra)"""

"\nSTACKED CHART\n\n\nvotos_apoio = fidelidade_deputado.groupby(['votacao_id', 'partido']).sum()['fidelidade'].to_frame().reset_index()\na = fidelidade_deputado.drop_duplicates(['votacao_id', 'partido'])[['votacao_id', 'partido', 'tam_bancada']]\nvotos = a.merge(votos_apoio, on=['votacao_id', 'partido'])\nvotos['contra'] = votos.apply(lambda x:  x['tam_bancada'] - x['fidelidade'], axis=1)\nvotos = votos.rename(columns= {'fidelidade': 'a_favor'})\nvotos_favor = votos.pivot(index='votacao_id', columns='partido', values='a_favor')\nvotos_contra = votos.pivot(index='votacao_id', columns='partido', values='contra')\ndef add_date(df):\n    df = df.reset_index().merge(votacao_id_table[['votacao_id', 'datavotacao']], on='votacao_id')\n    df.index = df['datavotacao']\n    del df['datavotacao']\n    del df['votacao_id']\n    df.sort_index(inplace=True)\n    return df\n\nvotos_contra = add_date(votos_contra)\nvotos_favor  = add_date(votos_favor)\nvotos_contra = votos_contra.fillna(0)\ndef stack(